In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url_1'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//li[@class="item product product-item"]')
                
                # = = = = = = = = = = = = = = =
                
                list_sku = [part.xpath('./descendant::p[@class="sku-title"]/text()')[0].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_url = [part.xpath('./descendant::a[@class="product-item-link"]/@href')[0].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_src = [part.xpath('./descendant::div[@class="img-container"]/img/@src')[0].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Page': int(input_.loc[a, 'Page']),
                                        'No.': [i+1 for i in range(len(list_part))],
                                        'Sku': list_sku,
                                        'Url': list_url,
                                        'Src': list_src})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Page': int(input_.loc[a, 'Page']),
                                     'Url_1': input_.loc[a, 'Url_1']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url_1'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Page', 'No.'], ascending=[True, True]).reset_index(drop=True)
output_correct['No.'] = [i+1 for i in range(len(output_correct))]
output_correct.to_excel('./menu-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Page'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./menu_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：4145

[ok] - https://www.centricparts.com/partFinder/page/index/?p=1&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4138] - [当前时间：10:01:13]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=4&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4137] - [当前时间：10:01:13]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=3&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4136] - [当前时间：10:01:13]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=7&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4135] - [当前时间：10:01:13]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=2&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4134] - [当前时间：10:01:14]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=5&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4133] - [当前时间：10:01:14]

[ok] - https://www.centricparts.com/partFi

[ok] - https://www.centricparts.com/partFinder/page/index/?p=55&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4086] - [当前时间：10:01:46]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=56&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4085] - [当前时间：10:01:47]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=59&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4084] - [当前时间：10:01:49]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=57&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4083] - [当前时间：10:01:49]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=54&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4082] - [当前时间：10:01:49]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=58&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4081] - [当前时间：10:01:50]

[ok] - https://www.centricparts.com/partFinder

[ok] - https://www.centricparts.com/partFinder/page/index/?p=106&product_list_limit=25&product_list_order=part_number
[尝试次数：2] - [剩余数量：4034] - [当前时间：10:02:26]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=102&product_list_limit=25&product_list_order=part_number
[尝试次数：2] - [剩余数量：4033] - [当前时间：10:02:28]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=111&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4032] - [当前时间：10:02:29]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=109&product_list_limit=25&product_list_order=part_number
[尝试次数：3] - [剩余数量：4031] - [当前时间：10:02:29]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=107&product_list_limit=25&product_list_order=part_number
[尝试次数：3] - [剩余数量：4030] - [当前时间：10:02:29]

[ok] - https://www.centricparts.com/partFinder/page/index/?p=112&product_list_limit=25&product_list_order=part_number
[尝试次数：1] - [剩余数量：4029] - [当前时间：10:02:30]

[ok] - https://www.centricparts.com/part

KeyboardInterrupt
2024-04-01T02:02:56Z


KeyboardInterrupt: 